In [ ]:
#..........................案例1:
#.......................... 降维
#......维数灾难（curse of dimentionality）：维度多会导致分布稀疏，需要大量的实例
#......降维肯定会丢失一些信息,一般用来加速训练
#...降维技术:
# 投影（适合某一维度都在低维空间），
# 流行学：流形假设（manifold hypothesis），它认为大多数现实世界的高维数据集大都靠近一个更低维的流形。这种假设经常在实践中被证实，但不总是这样。
#...PCA(主成分分析，投影保留最大方差)
# SVD奇异分解的VT部分
from sklearn.decomposition import PCA
from sklearn.datasets import make_moons
X,y=make_moons(n_samples=100)
pca=PCA(n_components=2)
X2D=pca.fit_transform(X) # 会自动中心化，2维
print('方差解释率：',pca.explained_variance_ratio_)
pca=PCA(n_components=0.95) # 0-1表示保留多少比率的方差
X_reduced=pca.fit_transform(X)
#...增量PCA
# 将训练集分批，然后计算，适用于数据集较大
from sklearn.decomposition import IncrementalPCA
import numpy as np
n_batches=100
inc_pca=IncrementalPCA(n_components=2) # 要低于总维度
for X_batch in np.array_split(X,n_batches):
    inc_pca.partial_fit(X_batch) # 对每个都进行partial_fit
X_mnist_reduced=inc_pca.transform(X)
# 另一种写法
filename=''
n=1000
X_mm=np.memmap(filename,dtype='float32',mode='readonly',shape=(n,2))
# memmap允许您操作存储在磁盘上二进制文件中的大型数组，就好像它完全在内存中
batch_size=n//n_batches
inc_pca=IncrementalPCA(n_components=154,batch_size=batch_size)
inc_pca.fit(X_mm)
#...随机PCA
rnd_pca=PCA(n_components=154,svd_solver='randomized')
X_reduced=rnd_pca.fit_transform(X)

In [10]:
#...核PCA
# 核技巧，一种将实例隐式映射到非常高维空间（称为特征空间）的数学技术
# 可以执行复杂的非线性投影来降低维度
from sklearn.decomposition import KernelPCA
rbf_pca=KernelPCA(n_components=2,kernel='rbf',gamma=0.04)
X_reduced=rbf_pca.fit_transform(X)
# 找最佳核和参数
# PCA和logit回归
from sklearn.model_selection import GridSearchCV  # CV格
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])
param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)
print('最佳参数:',grid_search.best_params_)
# 另一种寻找最佳参数方法：最小化重建前图像误差
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage) # 最小化误差

最佳参数: {'kpca__gamma': 0.03, 'kpca__kernel': 'rbf'}


0.24690435095920732

In [11]:
# LLE，局部线性嵌入，流形学技术的非线性降维
# 将流形较强的多维空间展开
# 处理大数据时较为吃力
from sklearn.manifold import LocallyLinearEmbedding
lle=LocallyLinearEmbedding(n_components=2,n_neighbors=10)
X_reduced=lle.fit_transform(X)
# 其他的降维方法：
# MDS 多维缩放
# lsomap
# t-SNE t-分布随机邻域嵌入
# LDA 线性判别分析，SVM之前，LDA是比较好的降维技术
